In [3]:
!pip install pyttsx3
!pip install SpeechRecognition
!pip install openai
!pip install requests
!pip install pyaudio
import speech_recognition as sr
import pyttsx3
import datetime

# Initialize TTS engine
engine = pyttsx3.init()
engine.setProperty('rate', 170)

# Set voice
def set_voice(language):
    voices = engine.getProperty('voices')
    for voice in voices:
        if language == 'ta' and ('Tamil' in voice.name or 'ta' in voice.id.lower()):
            engine.setProperty('voice', voice.id)
            return
        elif language == 'en' and ('English' in voice.name or 'en' in voice.id.lower()):
            engine.setProperty('voice', voice.id)
            return

# Speak and print
def speak(text, lang='en'):
    print(f">> {text}")
    set_voice(lang)
    engine.say(text)
    engine.runAndWait()

# Listen to voice input
def listen(lang_code='en-IN'):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        r.adjust_for_ambient_noise(source)
        try:
            audio = r.listen(source, timeout=6)
            command = r.recognize_google(audio, language=lang_code)
            print(f"You said: {command}")
            return command.lower()
        except sr.WaitTimeoutError:
            speak("You were silent. Please say again.", 'en')
        except sr.UnknownValueError:
            speak("Sorry, I couldn't understand. Please repeat.", 'en')
        except sr.RequestError:
            speak("Network error. Please check your connection.", 'en')
    return None

# Time/date response
def tell_time_and_date(command, lang='en'):
    now = datetime.datetime.now()
    if lang == 'ta':
        if "நேரம்" in command or "சமயம்" in command:
            speak(f"இப்போது நேரம் {now.strftime('%I:%M %p')}", 'ta')
        elif "தேதி" in command or "நாள்" in command:
            speak(f"இன்றைய தேதி {now.strftime('%B %d, %Y')}", 'ta')
        else:
            speak("நேரம் அல்லது தேதி கேட்டால் பதிலளிக்கிறேன்.", 'ta')
    else:
        if "time" in command:
            speak(f"The time is {now.strftime('%I:%M %p')}", 'en')
        elif "date" in command:
            speak(f"Today's date is {now.strftime('%B %d, %Y')}", 'en')
        else:
            speak("Please ask about the time or date.", 'en')

# Start assistant
speak("Hello! Welcome.", 'en')
speak("How can I assist you today?", 'en')
speak("Please say your preferred language: English or Tamil.", 'en')

# Voice-based language selection
while True:
    lang_choice = listen('en-IN')
    if lang_choice:
        if "tamil" in lang_choice or "தமிழ்" in lang_choice:
            user_lang = 'ta'
            recog_lang = 'ta-IN'
            speak("தமிழ் மொழி தேர்ந்தெடுக்கப்பட்டது. நேரம், தேதி, அல்லது exit என்று கேளுங்கள்.", 'ta')
            break
        elif "english" in lang_choice:
            user_lang = 'en'
            recog_lang = 'en-IN'
            speak("English selected. You can ask about the time, date or say exit.", 'en')
            break
        else:
            speak("Please say either 'English' or 'Tamil'.", 'en')

# Assistant Loop
while True:
    command = listen(recog_lang)
    if command:
        if "exit" in command or "வெளியேறு" in command:
            if user_lang == 'ta':
                speak("நன்றி. வெளியேறுகிறேன். பிறகு சந்திப்போம்!", 'ta')
            else:
                speak("Thank you. Goodbye!", 'en')
            break
        else:
            tell_time_and_date(command, lang=user_lang)

>> Hello! Welcome.
>> How can I assist you today?
>> Please say your preferred language: English or Tamil.
Listening...
You said: Tamil
>> தமிழ் மொழி தேர்ந்தெடுக்கப்பட்டது. நேரம், தேதி, அல்லது exit என்று கேளுங்கள்.
Listening...
You said: இப்போது நேரம் என்ன
>> இப்போது நேரம் 10:19 AM
Listening...
You said: இப்போது தேதி
>> இன்றைய தேதி May 16, 2025
Listening...
You said: வெளியேறு
>> நன்றி. வெளியேறுகிறேன். பிறகு சந்திப்போம்!
